In [29]:
import os
import sys

import pandas as pd
import json

In [30]:
import pandas as pd
import json

dataset_path = "/Users/alessandromolinarroet/Desktop/web_application/Scapers/arbeitnow_jobs.csv"
skill_file = ""

df = pd.read_csv(dataset_path)

In [ ]:
# generative model (a bit slow + not very good + only english) 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [32]:
job_description = """
We are hiring a Python backend engineer with experience in APIs, SQL, Docker,
Git, cloud services (AWS preferred), and Agile methodologies.
"""

prompt = f"Extract the list of required skills from this job description:\n{job_description}"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

- Python - SQL - Docker - Git - Cloud - Agile - Experience in Python - Python - Backend engineering


In [ ]:
## Prototype chuking + similiraty (could be good to reduce size of description if done better)

from sentence_transformers import SentenceTransformer, util
import numpy as np
import re

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def chunk_text(text, chunk_size=1000, overlap=20):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

SKILL_LIST = [ 
    "hard skills", "requirments" "name of skill", "your profile"
    ]

def similarity_chunks(text, similarity=SKILL_LIST, levels=5, top_k=3):

    chunks = chunk_text(text, chunk_size=5000, overlap=50)
    skill_vec = model.encode(similarity, normalize_embeddings=True)
    
    for level in range(levels):

        embeddings = model.encode(chunks, normalize_embeddings=True)
        sims = util.cos_sim(skill_vec, embeddings)  
        sims = sims.max(dim=0).values   
        top_idx = sims.argsort(descending=True)[:top_k]
        chunks = [chunks[i] for i in top_idx]

        initial_chunk = 5000
        min_chunk = 100
        decay = 0.5
        new_chunk_size = max(int(initial_chunk * (decay ** level)), min_chunk)

        refined = []
        for c in chunks:
            refined.extend(chunk_text(c, chunk_size=new_chunk_size, overlap=5))

        chunks = refined
    return chunks

if __name__ == "__main__":
    

    jd = """Your Impact

As part of Solution Engineering at Tacto, you'll bridge our powerful platform with measurable customer value through data expertise. Working directly with customers, you'll understand their unique supply chain challenges and implement tailored technical solutions. By integrating customer procurement data and configuring the platform to match their processes, you'll drive adoption and showcase immediate ROI. This technical role combines data engineering with customer interaction – you'll not only build robust data pipelines but also guide customers through implementation, helping transform our technology into business impact for Europe's industrial backbone. Your work will directly influence customer success, product evolution, and our company's growth trajectory.

Your Tasks

Data Integration & Pipeline Development: Design and implement stable, scalable pipelines that transform customer procurement data into actionable insights within Tacto's platform.
Technical Onboarding Leadership: Drive the technical implementation process from kickoff to completion, working with customer IT teams to ensure smooth integration and rapid time-to-value.
Solution Configuration: Customize the Tacto platform to match each customer's unique requirements, applying best practices in procurement and supplier management.
Cross-Functional Collaboration: Work closely with Customer Success, Product, and Engineering to deliver integrated solutions and feed customer insights back into our product roadmap.
Continuous Improvement: Develop reusable components and standardized approaches that increase onboarding efficiency while maintaining quality and customer satisfaction.
Become AI Native: Proactively identify opportunities to leverage AI to accelerate your work, experiment with new approaches, and share successful workflows – contributing to Tacto's culture of high-velocity execution and innovation.

Your Profile

Now you may wonder what experiences and skills you need for this role. We believe that problem-solving, creativity, and drive are more important than tools that can be picked up. However, the following references will give a guideline of what experiences we think might be helpful.

You enjoy getting to know customers, understanding their needs and achieving results together with an interdisciplinary team
You have experience in programming with SQL
You are familiar with importing, cleansing, validating, transforming and exploring data
You have worked with BI tools such as PowerBI or Tableau
You have strong communication skills in both German and English

Who We Are

Tacto is an AI-driven procurement platform. Our mission is to strengthen Europe’s industrial “Mittelstand” by future proofing their supply chains. We streamline how companies buy, negotiate and manage procurement – we help them optimize their by far largest cost driver to stay competitive in the long-run. Trusted by hundreds of companies, Tacto transforms procurement into a strategic advantage.

Based in Munich, our team of builders, entrepreneurs and industry experts is creating technology that truly matters. We’re backed by some of the best investors in the world including Sequoia Capital, Index Ventures, Cherry Ventures, Visionaries Club, UVC Partners as well as the founders of Personio, GetYourGuide, Helsing & Forto.

Why It Matters

The industrial Mittelstand – 90,000+ small and medium-sized enterprises – forms Europe’s economic backbone, driving 25% of Germany’s GDP and providing over 30% of jobs. These hidden champions manufacture the physical world around us, from the machines that build our cities to the medical devices that save lives.

Today, a world of growing uncertainty and disruption is threatening their position. To stay competitive, the Mittelstand must unlock new efficiencies – and procurement is their largest, strategic lever to do so.

At Tacto, we deliver tangible cost savings and strategic resilience at a time when it’s needed most. Our work extends beyond technology – it’s about protecting the foundation of Europe’s economy for generations to come.

What We Offer

An environment where we enable you to have an impact and where we focus on outcomes, not time-tracking
A vibrant in-office culture in a beautiful office (at least 3 days/week)
26 + 4 vacation days per year (4 fixed “company rest days”)
Wellpass membership and access to Jobrad, for your health
Additional voluntary pension contribution and KITA subsidy, for you and your family’s future
Competitive salary & meaningful equity stake, so you are invested in our success

Tacto is an equal opportunities employer. We are committed to equal employment opportunity regardless of race, religion, sexual orientation, age, marital status, disability or gender identity. Please do not submit personal data revealing racial or ethnic origin, political opinions, religious or philosophical beliefs, trade union membership, data concerning your health, or data concerning your sexual orientation.
"""
    chunks = similarity_chunks(jd)
    for c in chunks:
        print(c)

“company rest days”) Wellpass membership and access to Jobrad, for your health Additional voluntary pension contribution and KITA subsidy, for you and your family’s future Competitive salary & meaningful equity stake, so you are invested in our success Tacto is an equal opportunities employer. We are committed to equal employment opportunity regardless of race, religion, sexual orientation, age, marital status, disability or gender identity. Please do not submit personal data revealing racial or ethnic origin, political opinions, religious or philosophical beliefs, trade union membership, data concerning your health, or data concerning your sexual orientation.
Your Impact As part of Solution Engineering at Tacto, you'll bridge our powerful platform with measurable customer value through data expertise. Working directly with customers, you'll understand their unique supply chain challenges and implement tailored technical solutions. By integrating customer procurement data and configuri

In [ ]:
# Fine tune very small model + adapter

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score 
model_name = "distilbert-base-uncased"
use_lora = True

label_list = ["O", "B-SOFT", "I-SOFT", "B-HARD", "I-HARD", "B-KNOW", "I-KNOW"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

dataset = [
    {
        "tokens": ["We", "are", "seeking", "a", "data", "analyst", "with", "strong", "Python", "and", "SQL", "skills", "and", "at", "least", "3", "years", "of", "experience", "."],
        "ner_tags": ["O","O","O","O","O","O","O","B-SOFT","B-HARD","O","B-HARD","O","O","O","O","B-KNOW","I-KNOW","O","I-KNOW","O"]
    },
    {
        "tokens": ["The", "candidate", "must", "have", "excellent", "communication", "skills", "and", "experience", "with", "Docker", "and", "Kubernetes", "."],
        "ner_tags": ["O","O","O","O","B-SOFT","I-SOFT","O","O","O","O","B-HARD","O","B-HARD","O"]
    },
    {
        "tokens": ["A", "Master", "'", "s", "degree", "in", "Computer", "Science", "is", "required", "with", "knowledge", "of", "TensorFlow", "and", "PyTorch", "."],
        "ner_tags": ["O","B-KNOW","I-KNOW","I-KNOW","I-KNOW","O","I-KNOW","I-KNOW","O","O","O","O","O","B-HARD","O","B-HARD","O"]
    },
    {
        "tokens": ["Looking", "for", "a", "project", "manager", "with", "strong", "leadership", "and", "problem-solving", "abilities", "and", "5", "+", "years", "managing", "technical", "teams", "."],
        "ner_tags": ["O","O","O","O","O","O","B-SOFT","I-SOFT","O","B-SOFT","I-SOFT","O","B-KNOW","I-KNOW","I-KNOW","I-KNOW","I-KNOW","I-KNOW","O"]
    },
    {
        "tokens": ["We", "need", "a", "backend", "engineer", "skilled", "in", "Node", ".", "js", "and", "REST", "APIs", "with", "AWS", "experience", "."],
        "ner_tags": ["O","O","O","O","O","O","O","B-HARD","I-HARD","I-HARD","O","B-HARD","I-HARD","O","B-HARD","I-HARD","O"]
    }
]

for item in dataset:
    item["ner_tags"] = [label2id[tag] for tag in item["ner_tags"]]

train_data = dataset
val_data = dataset[:1]

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        max_length=128,
        padding=False
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100) 
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx]) 
            else:
                label_ids.append(-100) 
            previous_word_idx = word_idx
        
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_dict["train"].column_names
)

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

if use_lora:
    print("Applying LoRA adapter...")
    lora_config = LoraConfig(
        task_type=TaskType.TOKEN_CLS,
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["q_lin", "v_lin"], 
        bias="none"
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()


data_collator = DataCollatorForTokenClassification(tokenizer)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

training_args = TrainingArguments(
    output_dir="./distilbert-skill-extraction",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4 if use_lora else 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5, 
    weight_decay=0.01,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=1,
    report_to="none",
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("Starting training...")
trainer.train()

output_path = "./distilbert-skill-extraction-final"
if use_lora:
    model.save_pretrained(output_path)
else:
    trainer.save_model(output_path)

tokenizer.save_pretrained(output_path)
print(f"Model saved to {output_path}")

eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

Map: 100%|██████████| 1/1 [00:00<00:00, 84.50 examples/s]
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/ks/r1n0ckqj7klc419d9ryqdxz40000gn/T/ipykernel_916/641266684.py:171: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Applying LoRA adapter...
trainable params: 152,839 || all params: 66,521,102 || trainable%: 0.2298
Starting training...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,1.746625,0.083333,0.200000,0.117647
2,No log,1.657096,0.000000,0.000000,0.000000
3,1.812500,1.587803,0.200000,0.200000,0.200000
4,1.812500,1.543345,0.000000,0.000000,0.000000
5,1.654700,1.524694,0.000000,0.000000,0.000000


/opt/anaconda3/envs/web_app/lib/python3.11/site-packages/transformers/trainer.py:4380: UserWarning: mtime may not be reliable on this filesystem, falling back to numerical ordering
  warnings.warn("mtime may not be reliable on this filesystem, falling back to numerical ordering")


Model saved to ./distilbert-skill-extraction-final



Final Evaluation Results:
eval_loss: 1.5878
eval_precision: 0.2000
eval_recall: 0.2000
eval_f1: 0.2000
eval_runtime: 0.0564
eval_samples_per_second: 17.7380
eval_steps_per_second: 17.7380
epoch: 5.0000


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from peft import PeftModel
import torch
import numpy as np

model_name = "distilbert-base-uncased"
output_path = "./distilbert-skill-extraction-final"
label_list = ["O", "B-SOFT", "I-SOFT", "B-HARD", "I-HARD", "B-KNOW", "I-KNOW"]
id2label = {i: label for i, label in enumerate(label_list)}

print("Loading base model...")
base_model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(output_path)

print(f"Loading LoRA adapter from {output_path}...")
model = PeftModel.from_pretrained(
    base_model,
    output_path
)
model.eval() 

print("Model loaded successfully. Ready for inference!")
print("-" * 30)

def predict_skills(text, model, tokenizer):
    inputs = tokenizer(
        text.split(),
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True
    )
    word_ids = inputs.word_ids()

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=-1).squeeze().cpu().numpy()

    final_labels = []
    
    for token_id, word_idx in enumerate(word_ids):
        if word_idx is not None and (word_idx == 0 or word_ids[token_id - 1] != word_idx):
            label = id2label[predictions[token_id]]
            final_labels.append((text.split()[word_idx], label))
    entities = [(word, label) for word, label in final_labels if label != 'O']
    
    return entities

test_sentence = "We need a strong leader with Python, teamwork, and 5 years of experience."
print(f"Input: {test_sentence}")

results = predict_skills(test_sentence, model, tokenizer)

print("\nPredicted Entities:")
if results:
    for word, label in results:
        print(f" - {word}: {label}")
else:
    print("No skills or knowledge entities detected.")

print("-" * 30)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading base model...
Loading tokenizer...
Loading LoRA adapter from ./distilbert-skill-extraction-final...
Model loaded successfully. Ready for inference!
------------------------------
Input: We need a strong leader with Python, teamwork, and 5 years of experience.

Predicted Entities:
 - need: B-HARD
 - Python,: I-KNOW
 - years: I-KNOW
------------------------------
